In [112]:
import numpy as np
import pandas as pd
from math import log

In [113]:
# read data and feature name from excel file
def read_data(path = None):
    df = pd.read_excel(path)

    # delete column 'nameid'
    df = df.drop(['nameid'], axis = 1)

    # discretize feature 'revenue'
    threshold = [0,10000,20000,30000,40000,50000]
    df['revenue'] = pd.cut(df['revenue'], threshold, labels = False)

    # fetch dataset and names of each feature
    dataset = df.values
    features = df.columns.values

    return dataset, features

In [114]:
# read dataset for training
train_dataset, train_features = read_data(r"train.xls")
train_data = pd.DataFrame(train_dataset, columns = train_features)

print(train_data)

     profession  education  house_loan  car_loan  married  child  revenue  \
0             5          1           0         0        1      1        0   
1             3          1           1         1        0      0        0   
2             2          3           1         0        1      0        1   
3             2          2           0         0        0      0        4   
4             4          2           0         1        0      1        1   
..          ...        ...         ...       ...      ...    ...      ...   
795           5          3           0         1        1      0        2   
796           5          5           0         0        1      1        3   
797           1          1           0         0        1      0        2   
798           3          5           0         1        0      0        1   
799           1          4           0         0        1      0        3   

     approve  
0          1  
1          0  
2          1  
3          1  


In [115]:
# read dataset for test
test_dataset, test_features = read_data(r"test.xls")
test_data = pd.DataFrame(test_dataset, columns = test_features)

print(test_data)

     profession  education  house_loan  car_loan  married  child  revenue  \
0             1          4           1         1        1      1        1   
1             2          1           0         0        1      0        4   
2             5          5           1         1        0      1        4   
3             5          5           1         0        1      0        1   
4             3          3           0         1        1      1        1   
..          ...        ...         ...       ...      ...    ...      ...   
195           3          2           1         1        0      0        3   
196           3          5           0         0        0      0        3   
197           4          2           1         1        0      1        1   
198           4          1           0         0        0      0        3   
199           5          4           0         1        1      1        1   

     approve  
0          0  
1          1  
2          0  
3          1  


In [116]:
# class for node of N-ary tree
class Tree_Node:
    def __init__(self, isleaf = True, label = None, feature_name = None, feature_num = None) -> None:
        self.isleaf = isleaf                # if this node is a leaf 
        self.label = label                  # label of the node if it is a leaf
        self.feature_name = feature_name    # name of its feature
        self.feature_num = feature_num      # number of its feature
        self.children = {}                  # children of this node
        self.result = {'label' : self.label, 'feature' : self.feature_num, 'children' : self.children}

    # fetch the info of this node
    def __repr__(self) -> str:
        return '{}'.format(self.result)
    
    # add a new child to this node
    def add_child(self, val, node) -> None:
        self.children[val] = node

    # predict the result base on given feature set
    def predict(self, features) -> int:
        # if this node turns out to be a leaf, just return its label
        if self.isleaf is True:
            return self.label
        
        # if the input feature value for this node is not valid
        if features[self.feature_num] not in self.children.keys():
            # fetch all valid values and pick a random one
            valid_feature_value_index = list(self.children.keys())
            x = np.random.randint(0, len(valid_feature_value_index) - 1)
            random_feature_value = valid_feature_value_index[x]
            
            # go on predicting on this randomly-chosen subtree
            return self.children[random_feature_value].predict(features)
        
        # otherwise just keep our prediction going
        return self.children[features[self.feature_num]].predict(features)

In [117]:
# class for decision tree
class Decision_Tree:
    def __init__(self, epsilon = 0.01) -> None:
        self.epsilon = epsilon              # threshold of gain ratio

    # calculate empirical entropy
    def calc_entropy(self, dataset) -> float:
        data_len = len(dataset)
        
        # count the frequency of each label
        label_count = {}
        for i in range(data_len):
            label = dataset[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        
        entropy = sum([(cnt / data_len) * log(cnt / data_len, 2) for cnt in label_count.values()])
        return -entropy

    # calculate empirical conditional entropy
    def calc_cond_entropy(self, dataset, col = 0) -> float:
        data_len = len(dataset)
        
        # partition dataset according to given feature
        subsets = {}
        for i in range(data_len):
            feature_val = dataset[i][col]
            if feature_val not in subsets:
                subsets[feature_val] = []
            subsets[feature_val].append(dataset[i])
        
        cond_entropy = sum([(len(set) / data_len) * self.calc_entropy(set) for set in subsets.values()])
        return cond_entropy

    # calculate the entropy of dataset about a certain feature
    def calc_axis_entropy(self, dataset, col = 0) -> float:
        data_len = len(dataset)
        
        # count the frequency of each value of this feature
        value_count = {}
        for i in range(data_len):
            value = dataset[i][col]
            if value not in value_count:
                value_count[value] = 0
            value_count[value] += 1

        col_entropy = sum([(cnt / data_len) * log(cnt / data_len, 2) for cnt in value_count.values()])
        return -col_entropy

    # calculate gain ratio
    def cal_gain_ratio(self, entropy, cond_entropy, col_entropy) -> float:
        # be careful that col_entropy may be 0
        critical_val = 1e-7
        if -critical_val <= col_entropy and col_entropy <= critical_val:
            return 0
        else:
            return (entropy - cond_entropy) / col_entropy 

    # find out the feature which holds the maxinum of gain ratio
    def gain_ratio_train(self, dataset):
        feature_count = len(dataset[0]) - 1
        
        # calculate the empirical entropy of the dataset
        entropy = self.calc_entropy(dataset)
        feature_ratio = []
        for i in range(feature_count):
            # calculate the gain ratio of each feature
            i_cond_entropy = self.calc_cond_entropy(dataset, col = i)
            i_col_entropy = self.calc_axis_entropy(dataset, col = i)
            i_gain_ratio = self.cal_gain_ratio(entropy, i_cond_entropy, i_col_entropy)
            feature_ratio.append((i, i_gain_ratio))
        
        best_feature = max(feature_ratio, key = lambda x: x[-1])
        return best_feature

    # input : dataset in DataFrame, feature set, threshold
    # output: decision tree
    def train(self, train_data):
        labels, features = train_data.iloc[:, -1], train_data.columns[:-1]
        
        # this node would be a leaf if there is only one kind of label
        if len(labels.value_counts()) == 1:
            return Tree_Node(isleaf = True, label = labels.iloc[0])

        # if the feature set is empty, another leaf, pick the major label for this node
        if len(features) == 0:
            return Tree_Node(isleaf = True, label = labels.value_counts().sort_values(ascending = False).index[0])

        # otherwise find out the feature which holds the maxinum of gain ratio
        best_feature, max_gain_ratio = self.gain_ratio_train(np.array(train_data))
        best_feature_name = features[best_feature]

        # if the gain ratio is less then threshold, another leaf, pick the major label for this node
        if max_gain_ratio < self.epsilon:
            return Tree_Node(isleaf = True, label = labels.value_counts().sort_values(ascending = False).index[0])

        # otherwise go on building the tree recursively
        node = Tree_Node(isleaf = False, feature_name = best_feature_name, feature_num = best_feature)

        # build a subtree for each value of this feature separately
        feature_values = train_data[best_feature_name].value_counts().index
        for val in feature_values:
            # pick all data of this feature value, and delete this used feature
            sub_train_data = train_data.loc[train_data[best_feature_name] == val].drop([best_feature_name], axis = 1)
            sub_tree = self.train(sub_train_data)
            node.add_child(val, sub_tree)
        
        return node

    # build the decision tree according to given dataset, and return the root node   
    def fit(self, train_data):
        self.root = self.train(train_data)
        return self.root

    def predict(self, test_data):
        return self.root.predict(test_data)

In [118]:
# build decision tree
dt_model = Decision_Tree()
root = dt_model.fit(train_data)

In [119]:
# use test dataset to estimate the performance of the model
result = []
for i in range(len(test_data)):
    example = test_data.iloc[i, :-1].values.tolist()
    result.append(root.predict(example))

# find the indicators that we need
TP, FP, FN, TN = 0, 0, 0, 0
for i in range(len(test_data)):
    if test_data.iloc[i, -1] == 1:
        if result[i] == 1:
            TP += 1
        else:
            FN += 1
    else:
        if result[i] == 1:
            FP += 1
        else:
            TN += 1

# calculate Precision Recall and F1-score
P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * TP / (2 * TP + FP + FN)

print("Precision: ", P)
print("Recall: ", R)
print("F1-score: ", F1)

Precision:  1.0
Recall:  0.6987951807228916
F1-score:  0.8226950354609929
